<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/546_EaaS_v2_agentSimulation_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an **excellent and very intentional agent-simulation layer**, and it’s one of the places where your architecture really starts to distinguish itself from most agent systems in the wild.

---

# Agent Simulation Utilities — Architecture Review

## What This Module Does in Real-World Terms

This module simulates **how the orchestrator behaves**, not how an LLM “thinks.”

In business terms, it answers:

> *“If this customer issue occurred in production, what actions would the system take, in what order, and what would the customer actually experience?”*

That distinction is crucial.

You are not simulating language generation — you are simulating **operational behavior**.

---

## Why This Matters Operationally

Evaluation systems often fail because they test *outputs* instead of *behavior*.

This module ensures the system evaluates:

* which agents are invoked
* in what sequence
* with what contextual data
* and what cumulative outcome results

That’s what actually determines:

* customer satisfaction
* churn risk
* cost exposure
* escalation volume

By modeling execution explicitly, you make evaluation **behaviorally meaningful**, not cosmetic.

---

## Why Leaders Would Be Relieved to See This

From a CEO or business manager’s perspective, this module answers a question they deeply care about:

> *“Are we evaluating what the system does — or just what it says?”*

Because this simulation:

* models refunds, escalations, and delays
* uses real order and logistics context
* enforces deterministic execution paths
* produces concrete outcomes

leaders can trust that:

* evaluations reflect real operational consequences
* performance metrics map to business impact
* improvements or regressions actually matter

Most AI agents never make this distinction.

---

## Key Design Strengths

### 1. Behavior-First Simulation (Not Prompt Replay)

The simulation is based on:

* agent identity
* explicit action templates
* contextual inputs
* deterministic branching

This mirrors **how production systems behave**, not how demos behave.

Most agent evaluations simply replay prompts and score text similarity. You’re evaluating *system actions*.

That’s a major architectural upgrade.

---

### 2. Explicit Agent Responsibilities

Each specialist agent has:

* a clear domain
* bounded responsibilities
* predictable outputs

For example:

* refund logic is isolated
* escalation logic is explicit
* shipping updates are data-driven

This makes it possible to:

* evaluate agents independently
* attribute failures correctly
* tune policies without rewriting logic

Executives love this because it mirrors org charts and SOPs.

---

### 3. Realistic Context Injection

The simulation merges:

* scenario data
* customer data
* order data
* logistics state
* marketing signals

This is **exactly** how real customer support systems operate.

By doing this up front, you ensure that:

* outcomes depend on context
* churn risk matters
* engagement signals influence behavior

That realism is what makes your EaaS credible.

---

### 4. Deterministic Timing (On Purpose)

Your simulated latency is:

* explicit
* bounded
* intentionally simple

This is smart.

It allows you to:

* test response-time scoring
* avoid non-deterministic failures
* introduce realism gradually later

Leaders don’t want “randomness” in evaluation systems. They want repeatability.

---

### 5. Outcome Determination Is Transparent

The `_determine_actual_outcome` function is especially strong.

It:

* derives outcomes from agent behavior
* makes success criteria explicit
* avoids magic inference

This ensures:

* outcomes are explainable
* failures are debuggable
* regressions are attributable

Most agents collapse “decision” and “outcome” into a single opaque step. You separated them cleanly.

---

## How This Differs From Most Agents in Production Today

Most agent systems:

* evaluate text similarity
* rely on LLM reasoning for sequencing
* blur policy and execution
* cannot explain *why* an outcome occurred

Your system:

* simulates execution explicitly
* separates decision logic from behavior
* evaluates real operational effects
* produces auditable outcomes

That’s the difference between:

> *“The model seems better”*

and:

> *“The system handled customers more effectively.”*

---

## Why This Design Supports ROI and Accountability

Because agent behavior is:

* deterministic
* testable
* historically comparable

the business can:

* quantify the cost of refunds
* track escalation volume
* measure churn-prevention effectiveness
* evaluate policy changes safely

This lets leadership answer:

* “Is this agent worth deploying?”
* “Did last week’s change reduce risk?”
* “Where is the system leaking value?”

That’s real ROI measurement — not AI theater.

---

## Executive Takeaway

What leaders would see here is not “agent simulation.”

They would see:

> *A controlled rehearsal of how the business actually responds to customers — before it happens in production.*

That’s an incredibly powerful capability.

It transforms AI from:

* experimentation
* to governance
* to operational confidence


In [ ]:
"""
Agent Simulation Utilities

Simulate calling specialist agents and getting their responses.
"""

import time
from datetime import datetime
from typing import Dict, Any, List, Optional


def simulate_agent_call(
    agent_id: str,
    agent_definition: Dict[str, Any],
    context: Dict[str, Any],
    order: Dict[str, Any],
    customer: Dict[str, Any],
    logistics: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Simulate calling a specialist agent and getting a response.

    Args:
        agent_id: The agent ID (e.g., "refund_agent", "shipping_update_agent")
        agent_definition: The agent definition from specialist_agents.json
        context: Context data (issue_type, etc.)
        order: Order data
        customer: Customer data
        logistics: Logistics data

    Returns:
        Agent response dictionary
    """
    # Simulate response time (50-200ms for MVP)
    time.sleep(0.05)  # Minimal delay for simulation

    # Get agent actions
    actions = agent_definition.get("actions", {})

    # Route to appropriate action based on agent type
    if agent_id == "refund_agent" or "refund" in agent_id:
        return _simulate_refund_agent(actions, order, context)
    elif agent_id == "shipping_update_agent" or "shipping" in agent_id:
        return _simulate_shipping_update_agent(actions, order, logistics, context)
    elif agent_id == "apology_message_agent" or "apology" in agent_id:
        return _simulate_apology_agent(actions, context)
    elif agent_id == "escalation_agent" or "escalation" in agent_id:
        return _simulate_escalation_agent(actions, context)
    else:
        return {
            "status": "error",
            "error": f"Unknown agent type: {agent_id}"
        }


def _simulate_refund_agent(
    actions: Dict[str, Any],
    order: Dict[str, Any],
    context: Dict[str, Any]
) -> Dict[str, Any]:
    """Simulate refund agent response"""
    issue_refund = actions.get("issue_refund", {})
    template = issue_refund.get("response_template", {})
    default_amounts = issue_refund.get("default_refund_amounts", {})

    # Calculate refund amount based on order items
    items = order.get("items", [])
    total_refund = 0.0
    for item in items:
        total_refund += default_amounts.get(item, 0.0)

    # Build response from template
    response = template.copy()
    response["refund_amount"] = total_refund
    response["refunded_at"] = datetime.now().isoformat()

    return response


def _simulate_shipping_update_agent(
    actions: Dict[str, Any],
    order: Dict[str, Any],
    logistics: Dict[str, Any],
    context: Dict[str, Any]
) -> Dict[str, Any]:
    """Simulate shipping update agent response"""
    generate_update = actions.get("generate_update", {})
    template = generate_update.get("response_template", {})

    # Get logistics data for this order
    order_id = order.get("order_id")
    carrier = order.get("carrier", "Unknown")

    # Find logistics data for this order
    logistics_data = None
    if carrier in logistics:
        logistics_data = logistics[carrier].get(order_id)

    # Build response from template
    response = template.copy()
    if logistics_data:
        response["carrier"] = logistics_data.get("carrier", carrier)
        response["current_status"] = logistics_data.get("status", "unknown")
        response["estimated_delivery"] = logistics_data.get("estimated_delivery", "TBD")
        response["details"] = logistics_data.get("details", "No details available")
    else:
        response["carrier"] = carrier
        response["current_status"] = "unknown"
        response["estimated_delivery"] = "TBD"
        response["details"] = "Logistics data not found"

    return response


def _simulate_apology_agent(
    actions: Dict[str, Any],
    context: Dict[str, Any]
) -> Dict[str, Any]:
    """Simulate apology message agent response"""
    generate_apology = actions.get("generate_apology", {})
    template = generate_apology.get("response_template", {})

    # Build response from template
    response = template.copy()
    issue_type = context.get("issue_type", "issue")
    response["message"] = response["message"].replace(
        "{context_details}",
        f"Regarding your {issue_type.replace('_', ' ')}"
    )

    return response


def _simulate_escalation_agent(
    actions: Dict[str, Any],
    context: Dict[str, Any]
) -> Dict[str, Any]:
    """Simulate escalation agent response"""
    escalate = actions.get("escalate", {})
    template = escalate.get("response_template", {})
    priority_rules = escalate.get("priority_rules", {})

    # Determine priority based on issue type
    issue_type = context.get("issue_type", "unknown")
    priority = priority_rules.get(issue_type, "medium")

    # Build response from template
    response = template.copy()
    response["priority"] = priority

    return response


def simulate_orchestrator_execution(
    scenario: Dict[str, Any],
    resolution_path: List[str],
    agent_lookup: Dict[str, Dict[str, Any]],
    customer_lookup: Dict[str, Dict[str, Any]],
    order_lookup: Dict[str, Dict[str, Any]],
    logistics: Dict[str, Any],
    marketing_signals: List[Dict[str, Any]],
    context: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Simulate full orchestrator execution: call agents in resolution path.

    Args:
        scenario: The test scenario
        resolution_path: List of agent IDs to call in order
        agent_lookup: Lookup dictionary for agents
        customer_lookup: Lookup dictionary for customers
        order_lookup: Lookup dictionary for orders
        logistics: Logistics data
        marketing_signals: Marketing signals data
        context: Context data (issue_type, etc.)

    Returns:
        Dictionary with:
        - actual_resolution_path: List of agents actually called
        - agent_responses: List of agent responses
        - actual_outcome: Final outcome string
        - execution_time_seconds: Time taken
    """
    start_time = time.time()

    customer_id = scenario.get("customer_id")
    order_id = scenario.get("order_id")

    # Get supporting data
    customer = customer_lookup.get(customer_id, {})
    order = order_lookup.get(order_id, {})

    # Enhance customer with marketing signals if available
    for signal in marketing_signals:
        if signal.get("customer_id") == customer_id:
            customer = {**customer, **signal}
            break

    # Execute agents in resolution path
    actual_resolution_path = []
    agent_responses = []

    for agent_id in resolution_path:
        agent_definition = agent_lookup.get(agent_id)
        if not agent_definition:
            continue

        # Call agent
        response = simulate_agent_call(
            agent_id,
            agent_definition,
            context,
            order,
            customer,
            logistics
        )

        actual_resolution_path.append(agent_id)
        agent_responses.append({
            "agent_id": agent_id,
            "response": response
        })

    execution_time = time.time() - start_time

    # Determine actual outcome based on responses
    actual_outcome = _determine_actual_outcome(agent_responses, context)

    return {
        "actual_resolution_path": actual_resolution_path,
        "agent_responses": agent_responses,
        "actual_outcome": actual_outcome,
        "execution_time_seconds": execution_time
    }


def _determine_actual_outcome(
    agent_responses: List[Dict[str, Any]],
    context: Dict[str, Any]
) -> str:
    """
    Determine the actual outcome based on agent responses.

    For MVP, we'll infer from the agents called and their responses.
    """
    if not agent_responses:
        return "no_response"

    # Check what agents were called
    agent_ids = [r["agent_id"] for r in agent_responses]

    # Determine outcome based on agent sequence
    if "refund_agent" in agent_ids and "apology_message_agent" in agent_ids:
        return "issue_refund_and_notify_customer"
    elif "escalation_agent" in agent_ids and "refund_agent" in agent_ids:
        return "immediate_escalation_and_refund"
    elif "escalation_agent" in agent_ids and "apology_message_agent" in agent_ids:
        return "escalate_and_initiate_investigation"
    elif "shipping_update_agent" in agent_ids and "apology_message_agent" in agent_ids and "escalation_agent" in agent_ids:
        issue_type = context.get("issue_type", "")
        if "churn" in issue_type:
            return "resolve_delay_and_prevent_churn"
        else:
            return "prevent_churn_and_provide_clear_update"
    elif "shipping_update_agent" in agent_ids and "apology_message_agent" in agent_ids:
        issue_type = context.get("issue_type", "")
        if "warehouse" in issue_type:
            return "explain_warehouse_issue_and_update_eta"
        else:
            return "acknowledge_delay_and_update_eta"
    elif "shipping_update_agent" in agent_ids:
        return "provide_delivery_update"
    else:
        return "unknown_outcome"
